# ML lifecycle automation tools

## MLflow

MLflow is an open-source platform developed by Databricks that allows ML practitioners, such as data scientists and ML engineerings, to automate the complete end-to-end machine learning lifecycle using a simple and intuitive code interface. It includes tools for tracking experiments, packaging code into reproducible runs and sharing and deploying models. Its open interface design can work with any language or platform, with clients in Python and Java, and is accessible through a REST API. 

MLflow provides APIs for logging parameters, code versions, metrics, and artifacts when running your machine learning code and for later visualizing the results. It also provides a library of reusable components for machine learning tasks, such as pre-processing data and training models, that can be packaged into a reusable, reproducible run.

In addition, MLflow integrates with a variety of machine learning libraries, including TensorFlow, PyTorch, and sci-kit-learn, allowing you to use the tools you are already familiar with while taking advantage of the advanced tracking and management capabilities of MLflow.

For this post, you will need the following prerequisites:

- The latest version of Docker installed in your machine. In case you don't have the latest version, please follow the instructions at the following URL: https://docs.docker.com/get-docker/.
- Access to a bash terminal (Linux or Windows).
- Access to a browser.
- Python 3.5+ installed.
- PIP installed.

## Why MLflow?

MLflow provides a single platform for everyday practitioners to handle the entire machine learning lifecycle, from iterating on model development to deploying it in a scalable and reliable environment that meets modern software system requirements.It facilitates the colaboration across roles and standarize the set of tools use a high level.

### Getting started with Mlflow

In the following code example, the Tracking API will be introduced. It would help us better understand how the MLflow API works and how it can be used to track our experiment's metrics, parameters, and artifacts during experimentation.


In [7]:
import os
import mlflow
from mlflow import log_metric, log_param, log_artifact
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [8]:
# Log a parameter (key-value pair)
log_param("param1", 5)

# Log a metric; metrics can be updated throughout the run
log_metric("foo", 1)
log_metric("foo", 2)
log_metric("foo", 3)

# Log an artifact (output file)
with open("output.txt", "w") as f:
    f.write("Hello world!")
log_artifact("output.txt")

Exception: '/home/jovyan/mlruns' does not exist.

We can observe that a new folder named `mlruns` was created along with some files.

In [ ]:
!cat /etc/*-release

In [ ]:
!tree mlruns -L 4

The mlruns folder is the default root directory for storing experiment runs and artifacts in the MLflow tracking component. The folder structure of the mlruns directory consists of multiple subdirectories, one for each run in your experiment.

Each run subdirectory is named after a unique run ID and contains the following files and directories:

- **meta.yaml:** A YAML file that contains metadata about the run, such as the run's status, start time, end time, and user-defined tags and parameters.
- **params:** A directory that contains YAML files for each parameter used in the run. Each file is named after the parameter name and contains its value.
- **tags:** A directory that contains YAML files for each tag applied to the run. Each file is named after the tag name and contains its value.
- **artifacts:** A directory that contains files or directories generated as artifacts during the run. The contents of the artifacts directory are determined by the user and may include models, data files, plots, or other outputs.

This structure allows for easy navigation of experiment runs, as well as efficient storage and retrieval of the metadata, parameters, tags, and artifacts associated with each run.

## Viewing the Tracking UI

By default, wherever you run your program, the tracking API writes data into files into an mlruns directory. You can then run MLflow’s Tracking UI to visualize your experiments metadata and information.

In [ ]:
!mlflow server -h 0.0.0.0 -p 4000

The experiments' metadata, runs information, parameters, and metrics can also be accessed programmatically by using the MLFlow client interface

In [ ]:
client = MlflowClient()

# list of experiments
experiments_list = client.search_experiments() # returns a list of mlflow.entities.Experiment
for experiment in experiments_list:
    print(experiment.name, experiment)

In [ ]:
# list of runs

runs_list = client.search_runs("Default") # returns a list of mlflow.entities.Experiment
for run in runs_list:
    print(run)

Let's review another example where we explore further the mlflow API.

In [ ]:
# create experiment
experiment_name = "Social NLP Experiments"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment:
    experiment_id = experiment.experiment_id
else:
    experiment_id = mlflow.create_experiment("Social NLP Experiments")
    
# ends the currently active run, if any, taking an optional run status.
mlflow.end_run()

# start a run
with mlflow.start_run(experiment_id=experiment_id, run_name="run1") as run1:
    mlflow.log_metric("m", 1.55)
    mlflow.set_tag("s.release", "1.1.0-RC")
    
    
# start another run
with mlflow.start_run(experiment_id=experiment_id, run_name="run2") as run2:
    mlflow.log_metric("m", 2.50)
    mlflow.set_tag("s.release", "1.2.0-GA")
    
# Search all runs under experiment id and order them by
# descending value of the metric 'm'
client = MlflowClient()
runs = client.search_runs(experiment_id, order_by=["metrics.m DESC"])
for r in runs:
    print(f"run_id: {r.info.run_id}")
    print(f"lifecycle_stage: {r.info.lifecycle_stage}")
    print(f"metrics: {r.data.metrics}")

    # Exclude mlflow system tags
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    print(r"tags: {tags}")


# Delete the first run
client.delete_run(run_id=run1.info.run_id)

# Search only deleted runs under the experiment id and use a case insensitive pattern
# in the filter_string for the tag.
filter_string = "tags.s.release ILIKE '%rc%'"
runs = client.search_runs(experiment_id, run_view_type=ViewType.DELETED_ONLY,
                            filter_string=filter_string)


In [ ]:
def get_best_run(experiment_id, metric):
    # Connect to mflow using the default connection
    client = MlflowClient()
    
    # Get all the runs for the experiment
    runs = client.search_runs(experiment_id)
    
    # Find the run with the highest accuracy metric
    best_run = None
    best_metric_value = 0
    for run in runs:
        metric_value = run.data.metrics[metric]
        if metric_value > best_metric_value:
            best_metric_value = metric_value
            best_run = run
    # Return the best run
    return best_run

In [ ]:
get_best_run(experiment_id, metric="m")

## The Machine Learning Workflow

Machine learning requires experimenting with a wide range of datasets, data preparation steps, and algorithms to build a model that maximizes some target metric. Once you have built a model, you also need to deploy it to a production system, monitor its performance, and continuously retrain it on new data and compare with alternative models.

Being productive with machine learning can therefore be challenging for several reasons:

- **It’s difficult to keep track of experiments.** When you are just working with files on your laptop, or with an interactive notebook, how do you tell which data, code and parameters went into getting a particular result?
- **It’s difficult to reproduce code.** Even if you have meticulously tracked the code versions and parameters, you need to capture the whole environment (for example, library dependencies) to get the same result again. This is especially challenging if you want another data scientist to use your code, or if you want to run the same code at scale on another platform (for example, in the cloud).
- **There’s no standard way to package and deploy models.** Every data science team comes up with its own approach for each ML library that it uses, and the link between a model and the code and parameters that produced it is often lost.

Moreover, although individual ML libraries provide solutions to some of these problems (for example, model serving), to get the best result you usually want to try multiple ML libraries. MLflow lets you train, reuse, and deploy models with any library and package them into reproducible steps that other data scientists can use as a “black box,” without even having to know which library you are using.

## MLflow Components
MLflow provides three components to help manage the ML workflow:

- **MLflow Tracking** is an API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results. You can use MLflow Tracking in any environment (for example, a standalone script or a notebook) to log results to local files or to a server, then compare multiple runs. Teams can also use it to compare results from different users.

- **MLflow Projects** are a standard format for **packaging reusable data science code**. Each project is simply a directory with code or a Git repository, and uses a descriptor file or simply convention to specify its dependencies and how to run the code. For example, projects can contain a conda.yaml file for specifying a Python Conda environment. When you use the MLflow Tracking API in a Project, MLflow automatically remembers the project version executed (for example, Git commit) and any parameters. You can easily run existing MLflow Projects from GitHub or your own Git repository, and chain them into multi-step workflows.

- **MLflow Models** offer a convention for packaging machine learning models in multiple flavors, and a variety of tools to help you deploy them. Each Model is saved as a directory containing arbitrary files and a descriptor file that lists several “flavors” the model can be used in. For example, a TensorFlow model can be loaded as a TensorFlow DAG, or as a Python function to apply to input data. MLflow provides tools to deploy many common model types to diverse platforms: for example, any model supporting the “Python function” flavor can be deployed to a Docker-based REST server, to cloud platforms such as Azure ML and AWS SageMaker, and as a user-defined function in Apache Spark for batch and streaming inference. If you output MLflow Models using the Tracking API, MLflow will also automatically remember which Project and run they came from.

